In [1]:
!git clone https://github.com/mgrankin/over9000.git

Cloning into 'over9000'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 67 (delta 35), reused 41 (delta 19), pack-reused 0
Unpacking objects: 100% (67/67), done.


In [2]:
!git clone https://github.com/lessw2020/mish.git

fatal: destination path 'mish' already exists and is not an empty directory.


In [0]:
from fastai.vision import *

In [0]:
path = untar_data(URLs.IMAGEWOOF)

In [0]:
class Mish(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        x = x *( torch.tanh(F.softplus(x)))
        return x

In [0]:
act_fn = Mish() #nn.ReLU(inplace=True)

class Flatten(Module):
    def forward(self, x): return x.view(x.size(0), -1)

def init_cnn(m):
    if getattr(m, 'bias', None) is not None: nn.init.constant_(m.bias, 0)
    if isinstance(m, (nn.Conv2d,nn.Linear)): nn.init.kaiming_normal_(m.weight)
    for l in m.children(): init_cnn(l)

def conv(ni, nf, ks=3, stride=1, bias=False):
    return nn.Conv2d(ni, nf, kernel_size=ks, stride=stride, padding=ks//2, bias=bias)

def noop(x): return x

def conv_layer(ni, nf, ks=3, stride=1, zero_bn=False, act=True):
    bn = nn.BatchNorm2d(nf)
    nn.init.constant_(bn.weight, 0. if zero_bn else 1.)
    layers = [conv(ni, nf, ks, stride=stride), bn]
    if act: layers.append(act_fn)
    return nn.Sequential(*layers)

class ResBlock(Module):
    def __init__(self, expansion, ni, nh, stride=1):
        nf,ni = nh*expansion,ni*expansion
        layers  = [conv_layer(ni, nh, 3, stride=stride),
                   conv_layer(nh, nf, 3, zero_bn=True, act=False)
        ] if expansion == 1 else [
                   conv_layer(ni, nh, 1),
                   conv_layer(nh, nh, 3, stride=stride),
                   conv_layer(nh, nf, 1, zero_bn=True, act=False)
        ]
        self.convs = nn.Sequential(*layers)
        # TODO: check whether act=True works better
        self.idconv = noop if ni==nf else conv_layer(ni, nf, 1, act=False)
        self.pool = noop if stride==1 else nn.AvgPool2d(2, ceil_mode=True)

    def forward(self, x): return act_fn(self.convs(x) + self.idconv(self.pool(x)))

def filt_sz(recep): return min(64, 2**math.floor(math.log2(recep*0.75)))

class MXResNet(nn.Sequential):
    def __init__(self, expansion, layers, c_in=3, c_out=1000):
        stem = []
        sizes = [c_in,32,64,64]  #modified per Grankin
        for i in range(3):
            stem.append(conv_layer(sizes[i], sizes[i+1], stride=2 if i==0 else 1))
            #nf = filt_sz(c_in*9)
            #stem.append(conv_layer(c_in, nf, stride=2 if i==1 else 1))
            #c_in = nf

        block_szs = [64//expansion,64,128,256,512]
        blocks = [self._make_layer(expansion, block_szs[i], block_szs[i+1], l, 1 if i==0 else 2)
                  for i,l in enumerate(layers)]
        super().__init__(
            *stem,
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            *blocks,
            nn.AdaptiveAvgPool2d(1), Flatten(),
            nn.Linear(block_szs[-1]*expansion, c_out),
        )
        init_cnn(self)

    def _make_layer(self, expansion, ni, nf, blocks, stride):
        return nn.Sequential(
            *[ResBlock(expansion, ni if i==0 else nf, nf, stride if i==0 else 1)
              for i in range(blocks)])

def mxresnet(expansion, n_layers, name, pretrained=False, **kwargs):
    model = MXResNet(expansion, n_layers, **kwargs)
    if pretrained: 
        #model.load_state_dict(model_zoo.load_url(model_urls[name]))
        print("No pretrained yet for MXResNet")
    return model

me = sys.modules[__name__]
for n,e,l in [
    [ 18 , 1, [2,2,2 ,2] ],
    [ 34 , 1, [3,4,6 ,3] ],
    [ 50 , 4, [3,4,6 ,3] ],
    [ 101, 4, [3,4,23,3] ],
    [ 152, 4, [3,8,36,3] ],
]:
    name = f'mxresnet{n}'
    setattr(me, name, partial(mxresnet, expansion=e, n_layers=l, name=name))

In [0]:
data = (ImageList.from_folder(path).split_by_folder(valid='val')
            .label_from_folder().transform(([flip_lr(p=0.5)], []), size=128)
            .databunch(bs=64, num_workers=2)
            .presize(128, scale=(0.35,1))
            .normalize(imagenet_stats))

In [12]:
%cd over9000/

/content/over9000


In [0]:
from over9000 import *

In [14]:
Over9000

<function over9000.Over9000>

In [0]:
bs_rat = 64/256
opt_func=partial(Over9000, betas = (0.9,0.99), eps=1e-6)

In [0]:
 n = len(learn.data.train_dl)

In [0]:
 anneal_start = int(n*5*0.7)

In [0]:
from fastai.script import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.distributed import *

In [0]:
TrainingPhase

In [0]:
lr = 1e-2

In [0]:
learn = Learner(data, mxresnet50(c_out=10), wd=1e-2, opt_func=opt_func,
                 metrics=[accuracy, top_k_accuracy],
                 bn_wd=False, true_wd=True,
                 loss_func=LabelSmoothingCrossEntropy())

In [0]:
res = []
num_epoch=5
for x in range(5):
  learn = Learner(data, mxresnet50(c_out=10), wd=1e-2, opt_func=opt_func,
                 metrics=[accuracy, top_k_accuracy],
                 bn_wd=False, true_wd=True,
                 loss_func=LabelSmoothingCrossEntropy())
  n = len(learn.data.train_dl)
  anneal_start = int(n*5*0.7)
  phase0 = TrainingPhase(anneal_start).schedule_hp('lr', lr)
  phase1 = TrainingPhase(n*5 - anneal_start).schedule_hp('lr', lr, anneal=annealing_cos)
  phases = [phase0, phase1]
  sched = GeneralScheduler(learn, phases)
  learn.callbacks.append(sched)
  learn.fit(num_epoch)
  
  loss, acc, topk = learn.validate()
  res.append(acc.numpy())

In [33]:
np.mean(res)

0.732

In [34]:
np.std(res)

0.009465724

In [35]:
res

[array(0.718, dtype=float32),
 array(0.73, dtype=float32),
 array(0.738, dtype=float32),
 array(0.746, dtype=float32),
 array(0.728, dtype=float32)]